# 改进

- 用iterative-stratification的CV策略线下GPU训练
- 直接加载模型即可
- 直接预测

In [ ]:
!pip install ../input/iterative-stratification/iterative_stratification-0.1.6-py3-none-any.whl

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit
from category_encoders import CountEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss

import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputClassifier


import os
import warnings
warnings.filterwarnings('ignore')

# Framing as a binary classification problem

In this notebook I create a baseline model using XGBoost and framing the problem as a n-binary classification problems (where n=206 and is the total number of classes). I make use of the [MultiOutputClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html#sklearn.multioutput.MultiOutputClassifier) wrapper in sklearn.

This has the advantages that :
- You can use models capable only of binary classification
- It is easy to implement

But has the disadvantages that:
- You lose any correlation between labels which could be useful to the model
- You need to train *n* models and is therefore slow



Updates (started version 9)
- v9: 
    - dropped ctl_vehicle instances in-fold, kept in validation 


In [ ]:
SEED = 20201
NFOLDS = 10
DATA_DIR = '/kaggle/input/lish-moa/'
np.random.seed(SEED)

In [ ]:
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')

test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

# drop id col
X = train.iloc[:,1:].to_numpy()
X_test = test.iloc[:,1:].to_numpy()
y = targets.iloc[:,1:].to_numpy() 

In [ ]:
classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))

clf = Pipeline([('encode', CountEncoder(cols=[0, 2])),
                ('classify', classifier)
               ])

In [ ]:
params = {'classify__estimator__colsample_bytree': 0.6522,
          'classify__estimator__gamma': 3.6975,
          'classify__estimator__learning_rate': 0.0503,
          'classify__estimator__max_delta_step': 2.0706,
          'classify__estimator__max_depth': 10,
          'classify__estimator__min_child_weight': 31.5800,
          'classify__estimator__n_estimators': 166,
          'classify__estimator__subsample': 0.8639
         }

_ = clf.set_params(**params)


## Train the model

Framing this problem as a binary classification problem has the disadvantage that you need to train as many models as you have classes. For this problem this means training 206 models per fold, for the large number of features included in this dataset this may take a long time...

In [ ]:
xgb_path = '../input/moa-xgb-10folds/'

In [ ]:
oof_preds = np.zeros(y.shape)
test_preds = np.zeros((test.shape[0], y.shape[1]))
oof_losses = []
# kf = KFold(n_splits=NFOLDS)
kf = MultilabelStratifiedKFold(n_splits=NFOLDS, random_state=202002, shuffle=True)
for fn, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print('Starting fold: ', fn)
    X_train, X_val = X[trn_idx], X[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]
    
    # drop where cp_type==ctl_vehicle (baseline)
    ctl_mask = X_train[:,0]=='ctl_vehicle'
    X_train = X_train[~ctl_mask,:]
    y_train = y_train[~ctl_mask]
    
#     print('...training')
#     clf.fit(X_train, y_train)
    
#     #Save XGB Model
#     # infer: https://www.kaggle.com/bogorodvo/starter-code-saving-and-loading-lgb-xgb-cb
    model_path = '{xgb_path}{name}_fold{fold}.bin'.format(xgb_path=xgb_path, name='xgb_model', fold=str(fn))
#     print('...' + model_path)
#     pickle.dump(clf, open(model_path, "wb"))
    #load 
    print('...load model')
    clf = pickle.load(open(model_path, "rb"))
    
    val_preds = clf.predict_proba(X_val) # list of preds per class
    val_preds = np.array(val_preds)[:,:,1].T # take the positive class
    oof_preds[val_idx] = val_preds
    
    loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
    oof_losses.append(loss)
    preds = clf.predict_proba(X_test)
    preds = np.array(preds)[:,:,1].T # take the positive class
    test_preds += preds / NFOLDS
    
print(oof_losses)
print('Mean OOF loss across folds', np.mean(oof_losses))
print('STD OOF loss across folds', np.std(oof_losses))

In [ ]:
# set control train preds to 0
control_mask = train['cp_type']=='ctl_vehicle'
oof_preds[control_mask] = 0

print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(oof_preds)))

## Analysis of OOF preds


In [ ]:
# set control test preds to 0
control_mask = test['cp_type']=='ctl_vehicle'

test_preds[control_mask] = 0

In [ ]:
# create the submission file
sub.iloc[:,1:] = test_preds
sub.to_csv('submission.csv', index=False)